In [2]:
# 2017.11.29 Sheng Wan
# Most popular hotel algorithm
import numpy as np 
import pandas as pd
import seaborn as sns
from subprocess import check_output
print(check_output(["ls", "/Users/shengwan/Desktop"]).decode("utf8"))

BigData_Final_Project_Proposal_Template.pptx
CV-Sheng Wan.pages
CV-Sheng Wan.pdf
Company List.pages
Data Explore
Final draft.pages
Intermediate Pre.key
MSEE_2018_12_Sheng_Wan_sw3190.pdf
MSdata.mat
Resume-Sheng Wan.pdf
Screen Shot 2017-11-26 at 9.40.43 PM.png
Screen Shot 2017-12-10 at 1.06.23 PM.png
cover letter.pages
cover letter.pdf
destinations.csv
eclipse-workspace alias
func.php
practice2.pdf
predicted_with_pandas.csv
predicted_with_pandas.csv.zip
sample_submission.csv
test.csv
train.csv
uDemo.sql
uPunditMVP.sql
user_location_country+hotel_country.png



In [3]:
sns.set()
train = pd.read_csv('/Users/shengwan/Desktop/train.csv',
                    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],
                    chunksize=1000000)
aggs = []
print('-'*38)
for chunk in train:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
    print('.',end='')
print('')
aggs = pd.concat(aggs, axis=0)
aggs.head()

--------------------------------------
......................................


,srch_destination_id,hotel_cluster,sum,count
0,1,20,0.0,2
1,1,30,0.0,1
2,1,60,0.0,2
3,4,22,1.0,2
4,4,25,1.0,2


In [5]:
import pymysql
aggs.sort_values('srch_destination_id')

,srch_destination_id,hotel_cluster,sum,count
0,0,3,0.0,1
0,0,3,0.0,1
0,1,20,0.0,2
0,1,20,0.0,2
1,1,30,1.0,10
2,1,60,0.0,3
2,1,60,0.0,2
1,1,30,0.0,1
0,1,20,0.0,1
0,1,60,0.0,1


In [6]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,0,3,0.0,2.0,0.10
1,1,20,4.0,22.0,5.10
2,1,30,2.0,20.0,3.00
3,1,57,0.0,1.0,0.05
4,1,60,0.0,17.0,0.85


In [7]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

In [8]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
0,3
1,20 30 60 57
2,20 30 53 46 41
3,53 60
4,82 25 32 58 78


In [9]:
test = pd.read_csv('/Users/shengwan/Desktop/test.csv',
                    dtype={'srch_destination_id':np.int32},
                    usecols=['srch_destination_id'],)

In [10]:
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head()

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59


In [11]:
most_pop_all = agg.groupby('hotel_cluster')['relevance'].sum().nlargest(5).index
most_pop_all = np.array_str(most_pop_all)[1:-1]
most_pop_all

'91 48 42 59 28'

In [12]:
test.hotel_cluster.fillna(most_pop_all,inplace=True)

In [15]:
test.head()

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59


In [26]:
test.hotel_cluster.to_csv('predicted_with_pandas.csv',header=True, index_label='id')
